In [1]:
import os
# import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from setuptools import setup
from setuptools.command.install import install
from IPython.display import Markdown as md
from Bio import SeqIO
# from scipy import stats
import csv
from itertools import product

# Global Variables

In [2]:
# data_fp ='/ru-auth/local/home/abrenner/myscratch/helping_labmates/demultiplexing/'
fp = '/lustre/fs4/home/amansisido/scratch/RICC-Y/SSRP-Seq-2/fastq_19bp/Attempt1_BLC2fastq/'
# fastq_to_split = '/read1/Undetermined_S0_R1_001.fastq'
# fastq_to_split = ' Undetermined_S0_R2_001.fastq'

# Get barcodes

In [3]:
# df = pd.read_csv(data_fp+'umi_demultiplex.csv')

In [4]:
# barcode_combos_pre = (list(product(df['start_barcode'], df['end_barcode'])))

In [5]:
# barcode_combos = [barcode_combos_pre[i] + ('combo'+str(i)+'.fastq',) for i in range(0,len(barcode_combos_pre))]

In [6]:
# len(barcode_combos)

# Read in file with names

In [7]:
path_to_barcodes = '/ru-auth/local/home/abrenner/myscratch/helping_labmates/demultiplexing/'
df = pd.read_csv(path_to_barcodes+'umi_demultiplex_w_names.csv')
barcode_combos = list(df.itertuples(index=False, name=None))
# barcode_combos = [str(barcode_combos_pre[i][2]) + ('.fastq',) for i in range(0,len(barcode_combos_pre))]

In [8]:
barcode_combos

[('TTACCGACNNNNNNNNNNN', 'CGTATTCG', '1_5X_HBV_31U.fastq'),
 ('TCGTCTGANNNNNNNNNNN', 'TCAAGGAC', '1_5X_HBV_23U.fastq'),
 ('TTCCAGGTNNNNNNNNNNN', 'AAGCACTG', '1_5X_HBV_18U.fastq'),
 ('TACGGTCTNNNNNNNNNNN', 'GCAATGGA', '1_5X_HBV_13U.fastq'),
 ('AAGACCGTNNNNNNNNNNN', 'CAATCGAC', '1_7X_HBV_31U.fastq'),
 ('CAGGTTCANNNNNNNNNNN', 'GGCGTTAT', '1_7X_HBV_23U.fastq'),
 ('TAGGAGCTNNNNNNNNNNN', 'GTTAAGGC', '1_7X_HBV_18U.fastq'),
 ('TACTCCAGNNNNNNNNNNN', 'CCTATACC', '1_7X_HBV_13U.fastq'),
 ('AGTGACCTNNNNNNNNNNN', 'CTCCTAGA', '1_9X_HBV_31U.fastq'),
 ('AGCCTATCNNNNNNNNNNN', 'GTTACGCA', '1_9X_HBV_23U.fastq'),
 ('TCATCTCCNNNNNNNNNNN', 'CTAGCAAG', '1_9X_HBV_18U.fastq'),
 ('CCAGTATCNNNNNNNNNNN', 'ATCTCGCT', '1_9X_HBV_13U.fastq'),
 ('TTGCGAGANNNNNNNNNNN', 'GTGCCATA', '0X_HBV_23U.fastq'),
 ('GAACGAAGNNNNNNNNNNN', 'GGTGATTC', '0X_HBV_18U.fastq'),
 ('CGAATTGCNNNNNNNNNNN', 'CACCTTAC', '0X_HBV_13U.fastq'),
 ('GGAAGAGANNNNNNNNNNN', 'TTCTCTCG', '0X_HBV_10U.fastq'),
 ('TCGGATTCNNNNNNNNNNN', 'TAGTTGCG', 'HBV_0hr_25

# Read in FASTQ and save reads in the correct file

In [11]:
counter = 0
for read1v2 in range(1,3):# Delete existing fastqs to ensure you do not write into existing files
    print(read1v2)
    fastq_to_split = 'Undetermined_S0_R'+str(read1v2)+'_001.fastq'
    data_fp = '/ru-auth/local/home/abrenner/myscratch/helping_labmates/demultiplexing/read'+str(read1v2)+'/'
    for combo in barcode_combos:
    #     print(data_fp + combo[1])
        if not os.path.isfile(data_fp + combo[2]):
            continue
        os.remove(data_fp + combo[2]) 


    # Iterate through fastq of interest
    for seq_record in SeqIO.parse(fp+fastq_to_split, """fastq"""):
        counter = counter + 1
        # Grab the string of interest from the description
        desc_string = seq_record.description[-28:]
        # Iterate through all combos and evaluate in that combo matches the seq_record
        for combo in barcode_combos:
            # Assess start barcode 
            criteria1 = combo[0][:8] == desc_string[:8]
            # Assess end barcode
            criteria2= combo[1]== desc_string[-8:]
            if criteria1 & criteria2:
    #           # If the file does not exist yet, just save the seq_record to a new file with that name
                if not os.path.isfile(data_fp + combo[2]):
                    SeqIO.write(seq_record, data_fp + combo[2], "fastq")
                # If the files does exist, read it in and add onto it
                else:
                    sequences = [i for i in SeqIO.parse(data_fp + combo[2], """fastq""")]
                    sequences.append(seq_record)
                    SeqIO.write(sequences, data_fp + combo[2], "fastq")
    #             outfile.close()
    print(counter)

1


KeyboardInterrupt: 

You can run this in terminal to see how many lines per fastq